# Finding Good Average MaxCut Angles

In this example we find a set of angles for MaxCut which, on average, work well for [$G(n,p)$](https://en.wikipedia.org/wiki/Erdős–Rényi_model) graphs with the transverse field mixer. We then compare our angles against the "median angles" found by [Lotshaw et al.](https://arxiv.org/pdf/2102.06813.pdf).

In [2]:
using JuliQAOA, Graphs
using Optim, LineSearches
using Statistics
using Random

Random.seed!(1);

Our approach will be to find a single set of angles which maximize performance over a collection of 50 randomly generated graphs.

In [3]:
n = 7
num_graphs = 50

# generate graphs and calculate the MaxCut objective values across them all
graphs = [erdos_renyi(n, 0.5) for _ in 1:num_graphs]
obj_vals = [[maxcut(g,x) for x in states(n)] for g in graphs]

# generate transverse field mixer
mixer = mixer_x(n)

X mixer on 7-qubit states

We'll measure performance with the approximation ratio $\langle H_C\rangle/\text{max}(H_C)$, and we'll add an overall minus sign since `Optim.jl` only does minimization.

In [4]:
# the function we are trying to minimize
f(x) = -sum([exp_value(x, mixer, vals)/maximum(vals) for vals in obj_vals])

f (generic function with 1 method)

We'll also need to define the gradient function. Optim wants gradients to come in the form `g!(G,x)`, where `g!` overwrites `G` with the gradient at `x`. And we'll add the `flip_sign` flag to account for minus sign in `f(x)`.

In [5]:
function g!(G, x)
    G .= 0
    tmpG = similar(G)
    for vals in obj_vals
        # calculate the gradient for the specific graph and store in tmpG
        grad!(tmpG, x, mixer, vals; flip_sign=true) 
        # add the results to the overall gradient G
        # remembering to divide by the maximum value to get the approximation ratio
        G .+= tmpG/maximum(vals)
    end
end

g! (generic function with 1 method)

Let's explore the angle space for 3 rounds by sampling 50 random points and doing gradient
descent (via BFGS).

In [6]:
p = 3
num_samples = 50

# store the BFGS results here
minimizers = []

for _ in 1:num_samples
    # start from a random point
    x0 = rand(2*p) .* 2π
    # we have found the `BackTracking` line search to perform best with QAOA
    minimizer = optimize(f, g!, x0, BFGS(linesearch=LineSearches.BackTracking()))
    push!(minimizers, minimizer)
end

In [9]:
# sort the minimizers by their minimum value
sort!(minimizers; by=x->minimum(x))
#print(minimizers[1])

# get the angles for the best minimizer
mean_angles = Optim.minimizer(minimizers[1])
#print(mean_angles)

# clean up the angles so they look nicer
# (they could be cleaned up further by exploiting symmetries of MaxCut, which we don't do here)
mean_angles = clean_angles(mean_angles, mixer, obj_vals[1])

 * Status: success

 * Candidate solution
    Final objective value:     -4.544708e+01

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.35e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.71e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.26e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 9.38e-16 ≰ 0.0e+00
    |g(x)|                 = 1.10e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    25
    f(x) calls:    55
    ∇f(x) calls:   26
[25.629717736512237, -13.780199829646065, -26.501960945886633, -49.88760197419078, 0.7222636880047494, 7.152357196632679]

6-element Vector{Float64}:
 0.49697650779389235
 5.069356091892693
 4.913965590011298
 0.37788048324591017
 0.7222636880047494
 0.8691718894530931

Let's compare against the "median angles" for this case found by Lotshaw et. al. across 50 new random graphs.

In [7]:
# taken from table VIII in 2102.06813
lotshaw_angles = [-0.15244, -0.10299, -0.06517, -0.12641, -0.24101, -0.27459] .* π

new_graphs = [erdos_renyi(n, 0.5) for _ in 1:num_graphs]
new_obj_vals = [[maxcut(g,x) for x in states(n)] for g in new_graphs]

lotshaw_approx_ratio = mean([exp_value(lotshaw_angles, mixer, vals)/maximum(vals) for vals in new_obj_vals])
our_approx_ratio = mean([exp_value(mean_angles, mixer, vals)/maximum(vals) for vals in new_obj_vals])

println("Lotshaw et. al. mean approximation ratio: $(lotshaw_approx_ratio)")
println("Our mean approximation ratio: $(our_approx_ratio)")

Lotshaw et. al. mean approximation ratio: 0.9087199002196616
Our mean approximation ratio: 0.9095472914503299
